# Aircraft maintainance problem - EA demo

In [ ]:
import pandas as pd
import datetime
import random
import sys
import os

## PlaneModel defines a set of useful classes

In [3]:
from  PlaneModel import job,aircraft,staff,problem

## Load Problem instance from Excel

Check that cells in the spreadsheet are a 'general' format. Numbers formatted as strings can cause issues

In [4]:
file = './Example_Case.xlsx'
planes = pd.read_excel(file,  sheet_name = 0)
work_packages = pd.read_excel(file, sheet_name = 1, index_col = 0)
people = pd.read_excel(file,  sheet_name = 2, index_col = 0)

## Populate Model

The data from Pandas (itself from Excel) is turned into a set of objects (as defined in the classes above)
The variable instance is an instance of Problem and contains collections of Staff, Aircraft and Jobs


In [5]:
instance = problem(people,planes,work_packages)

## Generate random genome

In [6]:

def randSol():
    instance.reset()
    availStaff = instance.getListStaff()
    
    allJobs = instance.getListJobs()
    sol = []
    for j in allJobs:
        theJob = instance.jobs[j]
        c=1
        for c in range(len(theJob.notAllocated)):
            
            rStaff = random.choice(availStaff)
            while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
                
            code= j+":"+str(c)
            t = [rStaff,code]
            sol.append(t)
            c=c+1
    random.shuffle(sol)
    return sol

## Evaluate Genome

In [7]:
def evaluate(sol):
    instance.reset()
    for g in sol:
        j=g[1].split(":")[0]
        instance.allocate(instance.staff[g[0]],instance.jobs[j])

    instance.validate()
    unalloc =instance.getUnallocated()
    over = 0
    countOver =0
    for ac in instance.aircraft:
        airc = instance.aircraft[ac]
        o = airc.over()
        over = over +o
        if o >0 :
            countOver = countOver + 1
        
    return (unalloc + over),unalloc,countOver,over


## Mutation function

In [8]:
def mutate(genome):
    
    ch = random.randint(0,3)
    if ch ==1:
        n = random.randint(0,len(genome)-1)
        availStaff = instance.getListStaff()
        
        j =  genome[n][1].split(":")[0]
        theJob = instance.jobs[j]
        rStaff = random.choice(availStaff)
        while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
        genome[n][0]= rStaff
        

    if ch==2:
        x = random.randint(0,len(genome)-1)
        y = random.randint(0,len(genome)-1)
        t = genome[x]
        genome.pop(x)
        genome.insert(y,t)
    if ch == 3:
        genome = timeMutate(genome)
    return genome


def timeMutate(genome):
    tl = datetime.timedelta(minutes=0)
    gene = None
    
    for c  in range(10):        
        x = random.randint(0,len(genome)-1)        
        g = genome[x]
        jb = g[1].split(':')[0]
        if instance.jobs[jb].duration > tl:
            tl = instance.jobs[jb].duration
            gene = x
    
    t = genome[gene]
    genome.pop(gene)
    genome.insert(0,t)
    return genome



## Copy genome

This is a deep copy

In [9]:
def copyG(genome):
    n = []
    for g in genome:
        n.append(g.copy())
    
    return n

# Simple EA

## Crossover (XO) function

In [10]:
def contains(genome, jCode):
    for j in genome:
        if j[1] == jCode:
            return True
    return False


def xo(pA,pB):
    if len(pA) != len(pB):
        print("Parent len mismatch")
  
    child = []
    for c in  range(len(pA)):
        if not contains(child,pA[c][1]):
                child.append(pA[c].copy())
        if not contains(child,pB[c][1]):
                child.append(pB[c].copy())
    
    return child


# EA

In [16]:
def tour(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] < p2[0]:
        return p1
    else:
        return p2
        
def rip(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] >  p2[0]:
        return p1
    else:
        return p2
          
    
pop_size = 1500 
budget = 100000

best = None

pop = []
for c in range(0,pop_size):

    i = randSol()
    f= evaluate(i)[0]
    p= (f,i)
    pop.append(p)
    if len(pop)==1:
        best=p
    
    
    if f < best[0]:
        best=p
        
print("Init")
print(best[0])
evals =0
while (evals < budget):
    evals = evals +1
    if random.choice([True,False]):
        parent = tour(pop)
        ng = copyG(parent[1])
    else:
        ng = xo(tour(pop)[1],tour(pop)[1])
        
    mutate(ng)
    nf = evaluate(ng)[0]
    child = (nf,ng)

    toGo = rip(pop)
    if toGo[0] > child[0]:
        pop.remove(toGo)
        pop.append(child)
        if child[0] < best[0]:
            best = (child[0],copyG(child[1])) 
            instance.reset()
            r = evaluate(best[1])
            print("Evals: " + str(evals))
            print("Improved (" +str(r[0])+") Missing staff = "+ str(r[1]) + " Late = "+ str(r[2]) + " Late mins = " + str(r[3]))
              
print("Done :" +str(best[0]))


Init
51
Evals: 2108
Improved (49) Missing staff = 39 Late = 1 Late mins = 10
Evals: 2877
Improved (48) Missing staff = 38 Late = 1 Late mins = 10
Evals: 4546
Improved (46) Missing staff = 36 Late = 1 Late mins = 10
Evals: 4704
Improved (41) Missing staff = 41 Late = 0 Late mins = 0
Evals: 4935
Improved (35) Missing staff = 35 Late = 0 Late mins = 0
Evals: 11158
Improved (34) Missing staff = 34 Late = 0 Late mins = 0
Evals: 11468
Improved (33) Missing staff = 33 Late = 0 Late mins = 0
Evals: 11715
Improved (32) Missing staff = 32 Late = 0 Late mins = 0
Evals: 13056
Improved (31) Missing staff = 31 Late = 0 Late mins = 0
Evals: 14152
Improved (30) Missing staff = 30 Late = 0 Late mins = 0
Evals: 15521
Improved (29) Missing staff = 29 Late = 0 Late mins = 0
Evals: 15639
Improved (28) Missing staff = 28 Late = 0 Late mins = 0
Evals: 21755
Improved (27) Missing staff = 27 Late = 0 Late mins = 0
Evals: 22203
Improved (26) Missing staff = 26 Late = 0 Late mins = 0
Evals: 26516
Improved (25) M

In [20]:
print("Final Result:")
instance.reset()
r = evaluate(best[1])
print("Fitness = " +str(r[0])+" Missing staff = "+ str(r[1]) + " Late Departures= "+ str(r[2]) + " Late mins = " + str(r[3]))
print("\nPlan:\n\n"+str(instance))

              

Final Result:
Fitness = 0 Missing staff = 0 Late Departures= 0 Late mins = 0

Plan:

101 ( 2025-04-01 06:05:00:2025-04-01 09:55:00Avail:2025-04-01 09:10:00  )
Scheduled Jobs 
	WP3 (2025-04-01 06:05:00-2025-04-01 07:05:00) * NA= S= AA10,AA16,
	WP1 (2025-04-01 08:30:00-2025-04-01 09:10:00) * NA= S= AA20,AA13,

102 ( 2025-04-01 07:10:00:2025-04-01 14:50:00Avail:2025-04-01 10:50:00  )
Scheduled Jobs 
	WP15 (2025-04-01 07:10:00-2025-04-01 08:30:00) * NA= S= AA20,AA12,AA19,
	WP2 (2025-04-01 10:10:00-2025-04-01 10:50:00) * NA= S= AA11,AA18,

103 ( 2025-04-01 08:50:00:2025-04-01 16:30:00Avail:2025-04-01 12:45:00  )
Scheduled Jobs 
	WP17 (2025-04-01 08:50:00-2025-04-01 10:10:00) * NA= S= BA5,BA3,AA11,
	WP7 (2025-04-01 10:10:00-2025-04-01 10:40:00) * NA= S= BA8,BA6,
	WP4 (2025-04-01 12:15:00-2025-04-01 12:45:00) * NA= S= AA2,AA6,

104 ( 2025-04-01 09:15:00:2025-04-01 16:55:00Avail:2025-04-01 12:45:00  )
Scheduled Jobs 
	WP5 (2025-04-01 09:15:00-2025-04-01 09:55:00) * NA= S= AA2,AA16,
	WP10 (2025